In [162]:
import requests
import unicodedata
import re
import pandas as pd 
from bs4 import BeautifulSoup as BS
import os
import numpy as np

os.chdir('/Users/oleteutloff/Desktop/Scraping_BERUFENET')

In [ ]:
# Create empty list with links
list_link_clean = list()

In [4]:
# Get job IDs for letter A page 1/7
start_link = 'https://berufenet.arbeitsagentur.de/berufenet/faces/index?path=null/sucheAZ&let=A'
    
link_s = str(start_link)
page = requests.get(start_link)    
soup = BS(page.content, 'html.parser')
    
main_clean = [soup.select('script[type="text/javascript"]')]
print(main_clean)

[[<script src="/berufenet/adf/jsLibs/Locale1_2_12_3_3.js" type="text/javascript"></script>, <script type="text/javascript">var AdfAssert = new Object();AdfAssert.DEBUG = false;</script>, <script src="/berufenet/afr/partition/unknown/n/default/opt/boot-11.1.1.9.0-0151.js" type="text/javascript"></script>, <script type="text/javascript">AdfLogger.DEFAULT_LEVEL=AdfLogger.OFF;(AdfLogger.LOGGER=new AdfLogger('global', new AdfBufferedLogWriter())).register();var checkAgentSupport = true;AdfAgent._UNSUPPORTED_BROWSER_ALERT="Sie verwenden einen nicht unterstützten Browser. Genauere Informationen zu den unterstützten Browserversionen finden Sie in der Anwendungsdokumentation.";AdfAgent.AGENT = AdfAgent.getAgent(window);var lookAndFeel = new AdfDhtmlLookAndFeel(window);AdfAgent._LOG_WRITERS = "NativeLogWriter";AdfPage.PAGE = lookAndFeel.createPage();var page = AdfPage.PAGE;page.setBaseResourceUrl("/berufenet");page.__setAutomationEnabled(false);page.setAnimationEnabled(true);page.__setSessionIde

In [ ]:
# Get job IDs for letter A page 1/7
match = re.findall("'dkzID\\'\:(\d+)",str(main_clean[0][-2]))

In [283]:
def get_links(website):
    
    soup = BS(website)
    main_clean = [soup.select('script[type="text/javascript"]')]
    match = re.findall("'dkzID\\'\:(\d+)",str(main_clean[0][-2]))
    
    return match

In [163]:
# Get competences for letter A page 1/7
df_results = pd.DataFrame(columns=['ID','title','category','core_skills','further_skills'])

for i in match[1:3]:
    
    ID = str(i)
    link = "https://berufenet.arbeitsagentur.de/berufenet/faces/index?path=null/sucheAZ/kurzbeschreibung/berufkompetenzen&dkz="+ID+"&let=A"
    
    # Get the whole page
    page = requests.get(link)    
    soup = BS(page.content, 'html.parser')
    
    # select relevant elements from page
    title = [tr.get_text() for tr in soup.select('div[class="header-primary-text"]')]
    category = [tr.get_text() for tr in soup.select('div[class="header-secondary-text"]')]
    core_skills = [tr.get_text() for tr in soup.select('ul[id="b20-32_1_Content_1"]')]
    further_skills = [tr.get_text() for tr in soup.select('ul[id="b20-32_2_Content_2"]')]
    
    # Clean skills
    try:
        core_skills = list(filter(lambda a: a != '', core_skills[0].splitlines()))
    except:
        core_skills = np.nan
        
    try:
        further_skills = list(filter(lambda a: a != '', further_skills[0].splitlines()))
    except:
        further_skills = np.nan
    
    # Append results to df_results
    new_row = {'ID':ID, 'title':title[0], 'category':category[0], 'core_skills':core_skills,'further_skills':further_skills}
    df_results = df_results.append(new_row, ignore_index=True)
    
    print(i)
    print(df_results.shape)

93943
(1, 5)
8202
(2, 5)


In [249]:
# Get salary expectations for letter A page 1/7
df_salary = pd.DataFrame(columns=['ID','salary_min','salary_max'])

for i in match:
    
    ID = str(i)
    link = "https://berufenet.arbeitsagentur.de/berufenet/faces/index?path=null/sucheAZ/kurzbeschreibung/verdienstmoeglichkeiten&dkz="+ID+"&let=A"
    
    # Get the whole page
    page = requests.get(link)    
    soup = BS(page.content, 'html.parser')
    
    # select relevant elements from page
    salary = [tr.get_text() for tr in soup.select('div[id="b50-0_0_Content_1"]')]
    
    try:
        salary = re.findall('[0-9]+.[0-9]+',salary[0])

        if len(salary) == 0:
                salary_min = np.nan
                salary_max = np.nan

        if len(salary) == 1:
                salary_min = int(salary[0].replace('.', ''))
                salary_max = np.nan

        if len(salary) == 2:
                salary_min = int(salary[0].replace('.', ''))
                salary_max = int(salary[1].replace('.', ''))

        if len(salary) > 2:
                print("Problem with the salary")
                salary_min = "Problem"
                salary_max = "Problem"

    except:
        salary_min = np.nan
        salary_max = np.nan

    # Append results to df_results
    new_row = {'ID':ID, 'salary_min':salary_min,'salary_max':salary_max}
    df_salary = df_salary.append(new_row, ignore_index=True)
    
    print(i)
    print(df_salary.shape)

94380
(1, 3)
93943
(2, 3)
8202
(3, 3)
10154
(4, 3)
4710
(5, 3)
10251
(6, 3)
14475
(7, 3)
15647
(8, 3)
132516
(9, 3)
59161
(10, 3)
94099
(11, 3)
94097
(12, 3)
133352
(13, 3)
59233
(14, 3)
94180
(15, 3)
94368
(16, 3)
94082
(17, 3)
90071
(18, 3)
93756
(19, 3)
94200
(20, 3)
58714
(21, 3)
100520
(22, 3)
59533
(23, 3)
30255
(24, 3)
29414
(25, 3)
94216
(26, 3)
93885
(27, 3)
93798
(28, 3)
94177
(29, 3)
93737
(30, 3)


In [251]:
# Merging results
test = pd.merge(df_results,df_salary,on='ID',how='left')

# Scraping BERUFENET

This scraper collects the ID, name, required competences and salary expectations for all jobs on BERUFENET.

Next to information on jobs the scraper includes study programs and continuing education programs. These can be filtered via category.

The below solution uses Google Chrome and requires the selenium chromedriver. Please adapt the path for the driver accordingly (you can simply save it together with the code).

Mac users with Catalina or above need to enable the chromedriver. To do this open the terminal, navigate to the folder, and run the following command: 

xattr -d com.apple.quarantine chromedriver  

## Libraries

In [1]:
from selenium import webdriver
from string import ascii_uppercase
import requests
import unicodedata
import re
import pandas as pd 
from bs4 import BeautifulSoup as BS
import numpy as np

## Functions

In [2]:
# This functions extracts all job IDs from a given html source code

def get_IDs(website):
    
    soup = BS(website)
    main_clean = [soup.select('script[type="text/javascript"]')]
    match = re.findall("'dkzID\\'\:(\d+)",str(main_clean[0][-2]))
    
    return match

In [3]:
# This function collects job competences for one letter (A-Z) based on job IDs

def get_competences(df_IDs,letter):
    
    df_results = pd.DataFrame(columns=['ID','title','category','core_skills','further_skills'])
    
    for i in df_IDs['ID']:
        
        print(i)
        ID = str(i)
        link = "https://berufenet.arbeitsagentur.de/berufenet/faces/index?path=null/sucheAZ/kurzbeschreibung/berufkompetenzen&dkz="+ID+"&let="+letter
        
        # Get the whole page
        page = requests.get(link)    
        soup = BS(page.content, 'html.parser')

        # select relevant elements from page
        title = [tr.get_text() for tr in soup.select('div[class="header-primary-text"]')]
        category = [tr.get_text() for tr in soup.select('div[class="header-secondary-text"]')]
        core_skills = [tr.get_text() for tr in soup.select('ul[id="b20-32_1_Content_1"]')]
        further_skills = [tr.get_text() for tr in soup.select('ul[id="b20-32_2_Content_2"]')]

        # Clean skills
        try:
            core_skills = list(filter(lambda a: a != '', core_skills[0].splitlines()))
        except:
            core_skills = np.nan

        try:
            further_skills = list(filter(lambda a: a != '', further_skills[0].splitlines()))
        except:
            further_skills = np.nan

        # Append results to df_results
        new_row = {'ID':ID, 'title':title[0], 'category':category[0], 'core_skills':core_skills,'further_skills':further_skills}
        df_results = df_results.append(new_row, ignore_index=True)
        
    return df_results

In [4]:
# This function collects the salary expectation for jobs starting with one letter (A-Z) based on job IDs

def get_salary(df_IDs,letter):
    
    df_salary = pd.DataFrame(columns=['ID','salary_min','salary_max'])
    
    for i in df_IDs['ID']:
    
        print(i)
        ID = str(i)
        link = "https://berufenet.arbeitsagentur.de/berufenet/faces/index?path=null/sucheAZ/kurzbeschreibung/verdienstmoeglichkeiten&dkz="+ID+"&let="+letter

        # Get the whole page
        page = requests.get(link)    
        soup = BS(page.content, 'html.parser')

        # select relevant elements from page
        salary = [tr.get_text() for tr in soup.select('div[id="b50-0_0_Content_1"]')]

        try:
            salary = re.findall('[0-9]+.[0-9]+',salary[0])

            if len(salary) == 0:
                    salary_min = np.nan
                    salary_max = np.nan

            if len(salary) == 1:
                    salary_min = int(salary[0].replace('.', ''))
                    salary_max = np.nan

            if len(salary) == 2:
                    salary_min = int(salary[0].replace('.', ''))
                    salary_max = int(salary[1].replace('.', ''))

            if len(salary) > 2:
                    print("Problem with the salary")
                    salary_min = "Problem"
                    salary_max = "Problem"

        except:
            salary_min = np.nan
            salary_max = np.nan

        # Append results to df_salary
        new_row = {'ID':ID, 'salary_min':salary_min,'salary_max':salary_max}
        df_salary = df_salary.append(new_row, ignore_index=True)

    return df_salary

## Running the Scraper

In [5]:
### Collect all job IDs using Selenium ###

driver = webdriver.Chrome(executable_path='/Users/oleteutloff/Desktop/Scraping_BERUFENET/chromedriver')

# dataframe to store all IDs
df_IDs = pd.DataFrame(columns=['letter','ID'])

for letter in ascii_uppercase:
    
    url = 'https://berufenet.arbeitsagentur.de/berufenet/faces/index?path=null/sucheAZ&let='+letter
    
    # In case a letter does not have a link (there are no jobs for X)
    try: 
        driver.get(url)
    except:
        print('Madonna')
        continue # jump to the next letter
        
    page_number = 1

    while True:
        try:
            # get the clickable link with the respective number
            link = driver.find_element_by_link_text(str(page_number))
            # click the link
            link.click()
            # save the source code of this new page
            driver.get(driver.current_url)
            # extract the IDs from the page
            html = driver.page_source
            df_intermediate = pd.DataFrame(get_IDs(html), columns=['ID'])

            df_intermediate['letter'] = letter
            df_IDs = df_IDs.append(df_intermediate)
            
        except:
            print("End of Letter",letter)
            break
        page_number += 1

driver.close()

End of Letter A
End of Letter B
End of Letter C
End of Letter D
End of Letter E
End of Letter F
End of Letter G
End of Letter H
End of Letter I
End of Letter J
End of Letter K
End of Letter L
End of Letter M
End of Letter N
End of Letter O
End of Letter P
End of Letter Q
End of Letter R
End of Letter S
End of Letter T
End of Letter U
End of Letter V
End of Letter W
End of Letter X
End of Letter Y
End of Letter Z


In [6]:
# Exclude wrongly collected X
df_IDs = df_IDs[df_IDs['letter']!='X']
# save IDs
df_IDs.to_csv('/Users/oleteutloff/Desktop/Scraping_BERUFENET/job_IDs.csv', , index=False)

In [5]:
# Read in IDs
df_IDs = pd.read_csv('/Users/oleteutloff/Desktop/Scraping_BERUFENET/job_IDs.csv')

### Caveat

For letter X there are no jobs. Accessing this letter makes the browser jump to the default which is letter A. Therefor, the scraper incorrectly collects the IDs of letter A also for letter X. For now, I simply exclude letter X from df_IDs and therefor from the data subsequent data collection.

In [6]:
df_IDs = df_IDs[df_IDs['letter']!='X']

In [ ]:
### Get competences ###

# Create dataframe to store the results
df_results = pd.DataFrame(columns=['ID','title','category','core_skills','further_skills'])

for letter in ascii_uppercase:
    
    try: 
        # Get competences for all jobs starting with given letter
        df = get_competences(df_IDs[df_IDs['letter']==letter],letter)
        #print('this is intermediate \n',df)
        # Append to the main results dataframe
        df_results = df_results.append(df)
        #print('This is result \n',df_results)

        print('End of letter',letter)
        print(df_results.shape)
        
    except:
        print('There was an exception for letter:',letter)
        continue

94380
93943
8202
10154
4710
10251
14475
15647
132516
59161
94099
94097
133352
59233
94180
94368
94082
90071
93756
94200
58714
100520
59533
30255
29414
94216
93885
93798
94177
93737
93924
93863
93967
9063
9065
14640
94172
94234
100535
15615
13952
51027
5281
94283
94036
58681
93860
94360
14509
27308
14324
6773
29047
15164
2968
59169
7754
7070
58682
27476
14991
59171
94355
94297
9131
94382
59173
128340
130772
13982
5215
14801
9765
67308
6006
9791
58573
94306
94213
15706
93960
94016
58685
93604
94174
28504
7024
7023
34754
5291
58709


In [ ]:
### Get salary expectations ###

# Create dataframe to store the results
df_salary = pd.DataFrame(columns=['ID','salary_min','salary_max'])

for letter in ascii_uppercase:
    
    try:
        # Get competences for all jobs starting with given letter
        df = get_salary(df_IDs[df_IDs['letter']==letter],letter)

        # Append to the main results dataframe
        df_salary = df_salary.append(df)

        print('End of letter',letter)
        print(df_salary.shape)
        
    except:
        print('There was an exception for letter:',letter)
        continue

In [ ]:
df_results.shape

In [ ]:
# exclude observations that are studyprograms ('Studienfach')
df_results[df_results['category']!='Studienfach'].shape

In [ ]:
# count remaining NAs in core skills
df_results['core_skills'].value_counts(dropna=False)

In [ ]:
### Combine results and write to csv ###

# Merging results
test = pd.merge(df_results,df_salary,on='ID',how='left')

# writing to csv - ADAPT THE PATH according to your needs
#df_results.to_csv('/Users/oleteutloff/Desktop/Scraping_BERUFENET/berufenet.csv')

# OLD but keep for now

In [327]:
# Collect all job IDs using Selenium
driver = webdriver.Chrome(executable_path='/Users/oleteutloff/Desktop/Scraping_BERUFENET/chromedriver')
url = 'https://berufenet.arbeitsagentur.de/berufenet/faces/index?path=null/sucheAZ&let=A'
driver.get(url)
page_number = 1
results = []

while True:
    try:
        # get the clickable link with teh respective number
        link = driver.find_element_by_link_text(str(page_number))
        # click the link
        link.click()
        # save the source code of this new page
        driver.get(driver.current_url)
        # extract the IDs from the page
        html = driver.page_source
        results.extend(get_links(html))
    except:
        print("End of Letter A")
        break
    page_number += 1
    
driver.close()

30
60
90
120
150
180
188
End of Letter A
